# Superposition Kata Workbook, Part 2

The [Superposition Kata Workbook, Part 1](./Workbook_Superposition.ipynb) includes the solutions of kata tasks 1 - 7.  Part 2 continues the explanations for the rest of the tasks.

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [ ]:
%package Microsoft.Quantum.Katas::0.10.1911.1607

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## <a name="greenberger-horne-zeilinger"></a>Task 8. Greenberger-Horne-Zellinger state.

**Input:** N (N $\ge$ 1) qubits in the |0 ... 0$\rangle$ state (stored in an array of length N). <br>
**Goal:** Change the state of the qubits to GHZ state $\frac{1}{\sqrt2}$ (|0 ... 0$\rangle$ + |1 ... 1$\rangle$).

### Solution

On <a href="https://en.wikipedia.org/wiki/Greenberger–Horne–Zeilinger_state">Wikipedia</a>, we can read that a "Greenberger–Horne–Zeilinger state (GHZ state) is a certain type of entangled quantum state that involves at least three subsystems (particles)… it is a quantum superposition of all subsystems being in state 0 with all of them being in state 1… The GHZ state is a maximally entangled quantum state.

The simplest one is the 3-qubit GHZ state:
|GHZ$\rangle$ = <font size="4"> $\frac{|000\rangle + |111\rangle}{\sqrt2}$ </font>"

Do you see the resemblance to the Bell state expression <font size="2"> |$\Phi^{+}\rangle$ = $ \frac{1}{\sqrt2} \big( |00\rangle + |11\rangle \big)$ </font> which was discussed in task number 6?

<table align="left" style="background-color:white; border:0 solid; tr  { background-color:white; }">
<col width=600>
</tr>
<td style="text-align:left; font-size:14px; background-color:white; vertical-align:top; border:0">If you recall, the Bell state was obtained using this circuit:</td>    
    <tr>
</tr>
<td style="text-align:left; background-color:white; vertical-align:top; border:0"><img align="left" src="./img/Task8HadamardCNOTCircuit.png"/></td>    
    <tr>
</table>

<table align="left" style="background-color: white; border:0 solid; tr  { background-color:white; }">
<col width=800>
</tr>
<td style="text-align:left; font-size:14px; background-color:white; vertical-align:top; border:0"> As we move forward, let’s see if we can use this circuit as a building block. First, lets add a third qubit to the above circuit:</td>    
<td style="text-align:left; background-color:white; vertical-align:top; border:0">
    <tr>
</tr>
<td style="text-align:left; background-color:white; vertical-align:top; border:0"><img align="left" src="./img/Task8Hadamardand3rdqubitircuit.png"/></td>    
<tr>
</table>

<table align="left" style="background-color: white; border:0 solid; tr  { background-color:white; }">
<col width=800>
</tr>
<td style="text-align:left; font-size:14px; background-color:white; vertical-align:top; border:0">Comparing this state with the desired end state, we see that they differ only in the third (right most) qubit:</td>    
<td style="text-align:left; background-color:white; vertical-align:top; border:0">
    <tr>
</tr>
<td style="text-align:left; background-color:white; vertical-align:top; border:0"><img align="left" src="./img/Task8diffin3rdrightmostqubit.png"/></td>    
<tr>
</table>

<table align="left" style="background-color: white; border:0 solid; tr  { background-color:white; }">
    <col width=1000>
    <td style="text-align:left; vertical-align:top; font-size:14px;background-color:white; border:0">Applying a controlled NOT operation using the first (left most) qubit as the control bit and using the third (right most) qubit as the target qubit: 
</td>
    </tr>
    <tr>
    <td style="text-align:center; vertical-align:top; background-color:white; border:0"><img align="left" src="./img/Task8HadamardAndCNOTCircuit.png"/></td>  
 </tr>  
 <tr>
    <td style="text-align:center; vertical-align:top; background-color:white; border:0"><img align="left" src="./img/Task8CNOTFlip.png"/></td>  
 </tr> 
 </table>
 
 <table align="left" style="background-color: white; border:0 solid; tr  { background-color:white; }">
    <col width=1000>
    <td style="text-align:left; vertical-align:top; font-size:14px;background-color:white; border:0">And similarly, a circuit with four qubits:
</td>
    </tr>
    <tr>
    <td style="text-align:center; vertical-align:top; background-color:white; border:0"><img align="left" src="./img/Task84QubitCircuit.png"/></td>  
 </tr>  
 <tr>
    <td style="text-align:left; vertical-align:top; font-size:14px; background-color:white; border:0">would yield $\frac{1}{\sqrt2} (|0000\rangle + |1111\rangle$)</td>  
 </tr> 
 </table>

In [ ]:
%kata T08_GHZ_State_Test

// Task 8. Greenberger–Horne–Zeilinger state
// Input: N qubits in |0...0⟩ state.
// Goal: create a GHZ state (|0...0⟩ + |1...1⟩) / sqrt(2) on these qubits.
operation GHZ_State (qs : Qubit[]) : Unit {
    // Hint: N can be found as Length(qs).
    H(qs[0]);
    
    for (q in Rest(qs)) {
        CNOT(qs[0],q);
    }
}

[Return to task 8 of the Superposition kata.](./Superposition.ipynb#greenberger-horne-zeilinger)

## <a name="superposition-of-all-basis-vectors"></a>  Task 9. Superposition of all basis vectors.

**Input:** $N$ ($N \ge 1$) qubits in the $|0 \dots 0\rangle$ state.

**Goal:**  Change the state of the qubits to an equal superposition of all basis vectors $\frac{1}{\sqrt{2^N}} \big (|0 \dots 0\rangle + \dots + |1 \dots 1\rangle\big)$. For example, for N = 2 the final state should be  $\frac{1}{\sqrt{2}} * \big (|00\rangle + |01\rangle + |10\rangle + |11\rangle\big)$.

<br/>
<details>
  <summary>Need a hint? Click here</summary>
  Is this state separable?
</details>

### Solution
Putting some light on the definition of “equal superposition”, we refer to <a href="https://en.wikipedia.org/wiki/Hadamard_transform#Quantum_computing_applications">Wikipedia section on the Hadamard gate:</a>

It seems a solution might be to apply a Hadamard transform to every single qubit. <br>

\begin{align*}
   H(|0\rangle)&= \frac{1}{\sqrt2}\big(|0\rangle + |1\rangle\big)\\
   H(|0\rangle)H(|0\rangle)&= \frac{1}{\sqrt2} (|0\rangle + |1\rangle) \otimes \frac{1}{\sqrt2} (|0\rangle + |1\rangle)\\   
               &= \frac{1}{\sqrt{2^2}}\big(|00\rangle + |01\rangle+ |10\rangle+ |11\rangle\big)\\
   H(|0\rangle)H(|0\rangle)H(|0\rangle)&= \frac{1}{\sqrt{2^2}}\big(|00\rangle + |01\rangle+ |10\rangle+ |11\rangle\big) \otimes \frac{1}{\sqrt2}\big(|0\rangle + |1\rangle\big)\\
               &= \frac{1}{\sqrt{2^3}}\big(|000\rangle + |010\rangle+ |100\rangle+ |110\rangle+ |001\rangle + |011\rangle+ |101\rangle+ |111\rangle\big)\\
    \underset{N}{\underbrace{H(|0\rangle)H(|0\rangle)H(|0\rangle)}} &= \frac{1}{\sqrt{2^N}}  \underset{N}{\underbrace{\big(|0\cdots0\rangle\big)}}+\cdots+\underset{N}{\underbrace{\big(|1\cdots1\rangle\big)}}\\    
\end{align*}

Which produces the final desired state: <font size="3"> $\frac{1}{\sqrt{2^N}}\big(|0 \cdots 0\rangle + \cdots + |1 \cdots 1\rangle\big)$
 
<font size="3">Thus, the solution requires us to iterative over the qubit array to apply the Hadamard operation to each element as follows:

In [ ]:
%kata T09_AllBasisVectorsSuperposition_Test

// Task 9. Superposition of all basis vectors
// Input: N qubits in |0...0⟩ state.
// Goal: create an equal superposition of all basis vectors from |0...0⟩ to |1...1⟩
// (i.e. state (|0...0⟩ + ... + |1...1⟩) / sqrt(2^N) ).
operation AllBasisVectorsSuperposition (qs : Qubit[]) : Unit {
    for (q in qs) {
        H(q);
    }
}

[Return to task 9 of the Superposition kata.](./Superposition.ipynb#superposition-of-all-basis-vectors)

## <a name="superposition-of-all-even-or-all-odd-numbers"></a>Task 10. Superposition of all even or all odd numbers.

**Inputs:** 

1. $N$ ($N \ge 1$) qubits in the $|0 \dots 0\rangle$ state (stored in an array of length $N$).
2. A boolean `isEven`.

**Goal:**  Prepare a superposition of all *even* numbers if `isEven` is `true`, or of all *odd* numbers if `isEven` is `false`.  
A basis state encodes an integer number using [big-endian](https://en.wikipedia.org/wiki/Endianness) binary notation: state $|01\rangle$ corresponds to the integer $1$, and state $|10 \rangle$ - to the integer $2$.  

> For example, for $N = 2$ and `isEven = false` you need to prepare superposition $\frac{1}{\sqrt{2}} \big (|01\rangle + |11\rangle\big )$,  
and for $N = 2$ and `isEven = true` - superposition $\frac{1}{\sqrt{2}} \big (|00\rangle + |10\rangle\big )$.
</details>

### Solution

Let’s look at some examples of basis states to illustrate the binary numbering system. 

The 4 basis states on $N = 2$ qubits can be split in two columns, where the left column represents the basis states that form the required superposition state for `isEven = false` and the right column - the basis states that form the required superposition state for `isEven = true`:

<img src="./img/Task10_1.png" width="400">
 
If we do the same basis state split for $N = 3$ qubits, the pattern becomes more obvious:
 
<img src="./img/Task10_2.png" width="400">

The two leftmost qubits go through all possible basis states for `isEven = false` and for `isEven = true`, and the rightmost qubit stays in the $|1\rangle$ state for `isEven = false` and in the $|0\rangle$ state for `isEven = true`. 

A quick sanity check for $N = 4$ qubits re-confirms the pattern:
 
<img src="./img/Task10_3.png" width="400">
 
Again, the three leftmost qubits go through all possible basis states in both columns, and the rightmost qubit stays in the same state in each column. 

The solution is to put all qubits except the rightmost one into an equal superposition (similar to what we did in Task 9) and to set the rightmost qubit to $|0\rangle$ or $|1\rangle$ depending on the `isEven` flag, using the X operator to convert $|0\rangle$ to $|1\rangle$ if `isEven = false`.

In [ ]:
%kata T10_EvenOddNumbersSuperposition_Test

operation EvenOddNumbersSuperposition (qs : Qubit[], isEven : Bool) : Unit is Adj {
    let N = Length(qs);
    for (i in 0 .. N-2) {
        H(qs[i]);
    }
    // for odd numbers, flip the last bit to 1
    if (not isEven) {
        X(qs[N-1]);
    }
}

[Return to task 10 of the Superposition kata.](./Superposition.ipynb#superposition-of-all-even-or-all-odd-numbers)

*Solutions to the rest of the tasks coming up soon...*